In [1]:
import sys
import os
from glob import glob
sys.path.append('../')

In [2]:
from src.pipeline.deeplabv3.deeplabv3 import DeepLabV3Plus

In [3]:
model = DeepLabV3Plus(image_size=512, num_classes=9)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 518, 518, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 256, 256, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [5]:
!gdown https://drive.google.com/uc?id=1B9A9UCJYMwTL4oBEo4RZfbMZMaZhKJaz

zsh:1: no matches found: https://drive.google.com/uc?id=1B9A9UCJYMwTL4oBEo4RZfbMZMaZhKJaz


In [10]:
NUM_TRAIN_IMAGES = 2000
NUM_VAL_IMAGES = 50
DATA_DIR = "../supervised/Agriculture-Vision-2021/train/"

train_images = sorted(glob(os.path.join(DATA_DIR, "images/rgb/*")))[:NUM_TRAIN_IMAGES]
train_masks = sorted(glob(os.path.join(DATA_DIR, "masks/*")))[:NUM_TRAIN_IMAGES]

val_images = sorted(glob(os.path.join(DATA_DIR, "images/rgb/*")))[
    NUM_TRAIN_IMAGES : NUM_VAL_IMAGES + NUM_TRAIN_IMAGES
]
val_masks = sorted(glob(os.path.join(DATA_DIR, "masks/*")))[
    NUM_TRAIN_IMAGES : NUM_VAL_IMAGES + NUM_TRAIN_IMAGES
]


In [11]:
train_images

['../supervised/Agriculture-Vision-2021/train/images/rgb/11IE4DKTR_11556-9586-12068-10098.jpg',
 '../supervised/Agriculture-Vision-2021/train/images/rgb/11IE4DKTR_6121-684-6633-1196.jpg',
 '../supervised/Agriculture-Vision-2021/train/images/rgb/11TAQJ6ET_1124-8510-1636-9022.jpg',
 '../supervised/Agriculture-Vision-2021/train/images/rgb/11TAQJ6ET_1148-8089-1660-8601.jpg',
 '../supervised/Agriculture-Vision-2021/train/images/rgb/11TAQJ6ET_1288-961-1800-1473.jpg',
 '../supervised/Agriculture-Vision-2021/train/images/rgb/11TAQJ6ET_1422-8427-1934-8939.jpg',
 '../supervised/Agriculture-Vision-2021/train/images/rgb/11TAQJ6ET_1567-8177-2079-8689.jpg',
 '../supervised/Agriculture-Vision-2021/train/images/rgb/11TAQJ6ET_1749-948-2261-1460.jpg',
 '../supervised/Agriculture-Vision-2021/train/images/rgb/11TAQJ6ET_1995-8426-2507-8938.jpg',
 '../supervised/Agriculture-Vision-2021/train/images/rgb/11TAQJ6ET_2034-926-2546-1438.jpg',
 '../supervised/Agriculture-Vision-2021/train/images/rgb/11TAQJ6ET_2172

In [14]:
import tensorflow as tf

IMAGE_SIZE = 512
BATCH_SIZE = 32
def read_image(image_path, mask=False):
    image = tf.io.read_file(image_path)
    if mask:
        image = tf.image.decode_png(image, channels=1)
        image.set_shape([None, None, 1])
        image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
    else:
        image = tf.image.decode_png(image, channels=3)
        image.set_shape([None, None, 3])
        image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
        image = tf.keras.applications.resnet50.preprocess_input(image)
    return image

def load_data(image_list, mask_list):
    image = read_image(image_list)
    mask = read_image(mask_list, mask=True)
    return image, mask


def data_generator(image_list, mask_list):
    dataset = tf.data.Dataset.from_tensor_slices((image_list, mask_list))
    dataset = dataset.map(load_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    return dataset

In [15]:
train_dataset = data_generator(train_images, train_masks)
val_dataset = data_generator(val_images, val_masks)
print("Train Dataset:", train_dataset)
print("Val Dataset:", val_dataset)

Train Dataset: <BatchDataset element_spec=(TensorSpec(shape=(32, 512, 512, 3), dtype=tf.float32, name=None), TensorSpec(shape=(32, 512, 512, 1), dtype=tf.float32, name=None))>
Val Dataset: <BatchDataset element_spec=(TensorSpec(shape=(32, 512, 512, 3), dtype=tf.float32, name=None), TensorSpec(shape=(32, 512, 512, 1), dtype=tf.float32, name=None))>


In [17]:
from tensorflow import keras
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=loss,
    metrics=["accuracy"]
)
model.save('saved_deeplabv3')

INFO:tensorflow:Assets written to: saved_deeplabv3/assets


INFO:tensorflow:Assets written to: saved_deeplabv3/assets
